# Infer model on array

---

## Imports

In [1]:
import torch
import xbatcher
import xarray as xr
import numpy as np

## Toy data

In [2]:
data = xr.DataArray(
    data=np.random.rand(100, 100, 10),
    dims=("x", "y", "t")
)
data

<xarray.DataArray (x: 100, y: 100, t: 10)> Size: 800kB
array([[[0.80102402, 0.31982866, 0.25465188, ..., 0.31021283,
         0.03268302, 0.7941252 ],
        [0.4382327 , 0.60519493, 0.86055512, ..., 0.91560426,
         0.75151732, 0.045112  ],
        [0.56324046, 0.16672896, 0.95892792, ..., 0.75294859,
         0.62661621, 0.9507573 ],
        ...,
        [0.8942051 , 0.67152092, 0.5936959 , ..., 0.88955481,
         0.00104593, 0.25334781],
        [0.20207265, 0.89166716, 0.11930909, ..., 0.44454162,
         0.16769459, 0.13463857],
        [0.52366905, 0.77434724, 0.18922243, ..., 0.60982095,
         0.81261007, 0.35591865]],

       [[0.36699177, 0.29720945, 0.47093257, ..., 0.08379803,
         0.52233317, 0.62779805],
        [0.78408776, 0.29027363, 0.06707094, ..., 0.83199475,
         0.26807477, 0.93323152],
        [0.70058356, 0.83639276, 0.82536327, ..., 0.94036962,
         0.79252184, 0.83713594],
...
        [0.39766555, 0.35994714, 0.50271116, ..., 0.46796155,
         0.97140383, 0.51062791],
        [0.70055487, 0.81950773, 0.76781762, ..., 0.05735074,
         0.73039909, 0.17101624],
        [0.8547392 , 0.21515083, 0.40187337, ..., 0.4583594 ,
         0.53213994, 0.15382577]],

       [[0.6465284 , 0.88463142, 0.33543765, ..., 0.49702278,
         0.40990036, 0.42300881],
        [0.20363819, 0.02142565, 0.66128113, ..., 0.56253861,
         0.33799959, 0.16641441],
        [0.67934574, 0.38965227, 0.68431764, ..., 0.56820253,
         0.18640022, 0.111049  ],
        ...,
        [0.91296116, 0.6995538 , 0.50325961, ..., 0.40427649,
         0.14176453, 0.26158606],
        [0.5348746 , 0.67937558, 0.22366613, ..., 0.48889003,
         0.50030053, 0.1855764 ],
        [0.8714776 , 0.9552773 , 0.06885801, ..., 0.95366137,
         0.73931089, 0.90909682]]], shape=(100, 100, 10))
Dimensions without coordinates: x, y, t

## Simple model

In [3]:
class MeanAlongDim(torch.nn.Module):
    def __init__(self, ax):
        super(MeanAlongDim, self).__init__()
        self.ax = ax

    def forward(self, x):
        return torch.mean(x, self.ax)

## Batch generator, dataset

In [4]:
from xbatcher.loaders.torch import MapDataset

bgen = xbatcher.BatchGenerator(
    data,
    input_dims=dict(x=10, y=10),
    input_overlap=dict(x=5, y=5),
)

ds = MapDataset(bgen)

inp = next(iter(ds))

# Check the input/output size of the first example
print("Input shape:", inp.shape)

mad = MeanAlongDim(-1)
print("Output shape:", mad(inp).shape)

Input shape: torch.Size([10, 10, 10])
Output shape: torch.Size([10, 10])


In [5]:
assert torch.allclose(mad(inp), torch.mean(inp, -1))

## Inference function

In [6]:
%run ./functions.ipynb

In [17]:
out_size_dict = _get_output_array_size(
    bgen = ds.X_generator,
    output_tensor_dim = dict(y=10, x=10, t=5),
    new_dim = ["t"],
    resample_dim = ["y", "x", "t"]
)
print(out_size_dict)

{'y': 100, 'x': 100, 't': 5}


In [11]:
from functools import partial

In [21]:
get_array_size_partial = partial(_get_output_array_size, bgen=ds.X_generator)

In [18]:
test_cases = [
    {
        "name": "Same dims and same input sizes 1",
        "function_inputs": {
            "output_tensor_dim": dict(y=10, x=10),
            "new_dim": [],
            "resample_dim": ["x", "y"]
        },
        "expected_output": dict(y=100, x=100)
    },
    {
        "name": "New dim added",
        "function_inputs": {
            "output_tensor_dim": dict(y=10, x=10, t=5),
            "new_dim": ["t"],
            "resample_dim": ["y", "x"]
        },
        "expected_output": dict(y=100, x=100, t=5)
    }
]         

In [22]:
for i, test_case in enumerate(test_cases):
    true_output = get_array_size_partial(**test_case["function_inputs"])
    success = true_output == test_case["expected_output"]
    message = "passed" if success else "failed"
    print(f"Test case {i} {message}")

Test case 0 passed
Test case 1 passed
